<a href="https://colab.research.google.com/github/Adithyagaddam/collaboratory/blob/main/sample_code_multi_threads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import _thread
import time

def print_time( threadName, delay):
   count = 0
   while count < 5:
      time.sleep(delay)
      count += 1
      print ("%s: %s" % ( threadName, time.ctime(time.time()) ))

try:
   _thread.start_new_thread( print_time, ("Thread-1", 2, ) )
   _thread.start_new_thread( print_time, ("Thread-2", 4, ) )
except:
   print ("Error: unable to start thread")
while 1:
   pass

Thread-1: Fri May  6 07:19:41 2022
Thread-2: Fri May  6 07:19:43 2022
Thread-1: Fri May  6 07:19:43 2022
Thread-1: Fri May  6 07:19:45 2022
Thread-2: Fri May  6 07:19:47 2022
Thread-1: Fri May  6 07:19:47 2022
Thread-1: Fri May  6 07:19:49 2022
Thread-2: Fri May  6 07:19:51 2022
Thread-2: Fri May  6 07:19:55 2022
Thread-2: Fri May  6 07:19:59 2022


In [ ]:
import threading
import time
exitFlag = 0

class myThread (threading.Thread):
   def __init__(self, threadID, name, counter):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.counter = counter
   def run(self):
      print ("Starting " + self.name)
      print_time(self.name, self.counter, 5)
      print ("Exiting " + self.name)
def print_time(threadName, delay, counter):
   while counter:
      if exitFlag:
         threadName.exit()
      time.sleep(delay)
      print ("%s: %s" % (threadName, time.ctime(time.time())))
      counter -= 1

thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

thread1.start()
thread2.start()
thread1.join()
thread2.join()
print ("Exiting Main Thread")


Starting Thread-1
Starting Thread-2
Thread-1: Fri May  6 07:18:56 2022
Thread-1: Fri May  6 07:18:57 2022
Thread-2: Fri May  6 07:18:57 2022
Thread-1: Fri May  6 07:18:58 2022
Thread-1: Fri May  6 07:18:59 2022
Thread-2: Fri May  6 07:18:59 2022
Thread-1: Fri May  6 07:19:00 2022
Exiting Thread-1
Thread-2: Fri May  6 07:19:01 2022
Thread-2: Fri May  6 07:19:03 2022
Thread-2: Fri May  6 07:19:05 2022
Exiting Thread-2
Exiting Main Thread


In [ ]:
import threading
import time
# A list of items that are being produced.  Note: it is actually
# more efficient to use a collections.deque() object for this.
items = []
# A condition variable for items
items_cv = threading.Condition()
def producer():
    print ("I'm the producer")
    for i in range(5):
        with items_cv:          # Always must acquire the lock first
            items.append(i)     # Add an item to the list
            items_cv.notify()   # Send a notification signal
        time.sleep(1)
def consumer():
    print ("I'm a consumer", threading.currentThread().name)
    while True:
        with items_cv:           # Must always acquire the lock
            while not items:     # Check if there are any items
                items_cv.wait()  # If not, we have to sleep
            x = items.pop(0)     # Pop an item off
        print (threading.currentThread().name,"got", x)
        time.sleep(5)
cons = [threading.Thread(target=consumer)
        for i in range(10)]
for c in cons:
    c.setDaemon(True)
    c.start()
producer()

I'm a consumer I'm a consumer Thread-19
Thread-18I'm a consumer Thread-20

I'm a consumer Thread-21
I'm a consumer Thread-22
I'm a consumerI'm a consumer Thread-24
 I'm a consumer Thread-25
Thread-23
I'm a consumer Thread-26
I'm a consumer Thread-27I'm the producer

Thread-19 got 0
Thread-20 got 1
Thread-21 got 2
Thread-18 got 3
Thread-22 got 4


In [ ]:
import threading
import random
import time

#inheriting threading class in Thread module
class Philosopher(threading.Thread):
    running = True  #used to check if everyone is finished eating

 #Since the subclass overrides the constructor, it must make sure to invoke the base class constructor (Thread.__init__()) before doing anything else to the thread.
    def __init__(self, index, forkOnLeft, forkOnRight):
        threading.Thread.__init__(self)
        self.index = index
        self.forkOnLeft = forkOnLeft
        self.forkOnRight = forkOnRight

    def run(self):
        while(self.running):
            # Philosopher is thinking (but really is sleeping).
            time.sleep(30)
            print ('Philosopher %s is hungry.' % self.index)
            self.dine()

    def dine(self):
        # if both the semaphores(forks) are free, then philosopher will eat
        fork1, fork2 = self.forkOnLeft, self.forkOnRight
        while self.running:
            fork1.acquire() # wait operation on left fork
            locked = fork2.acquire(False) 
            if locked: break #if right fork is not available leave left fork
            fork1.release()
            print ('Philosopher %s swaps forks.' % self.index)
            fork1, fork2 = fork2, fork1
        else:
            return
        self.dining()
        #release both the fork after dining
        fork2.release()
        fork1.release()
 
    def dining(self):			
        print ('Philosopher %s starts eating. '% self.index)
        time.sleep(30)
        print ('Philosopher %s finishes eating and leaves to think.' % self.index)

def main():
    forks = [threading.Semaphore() for n in range(5)] #initialising array of semaphore i.e forks

    #here (i+1)%5 is used to get right and left forks circularly between 1-5
    philosophers= [Philosopher(i, forks[i%5], forks[(i+1)%5])
            for i in range(5)]

    Philosopher.running = True
    for p in philosophers: p.start()
    time.sleep(100)
    Philosopher.running = False
    print ("Now we're finishing.")
 

if __name__ == "__main__":
    main()


Philosopher 1 finishes eating and leaves to think.Philosopher 4 finishes eating and leaves to think.Philosopher 3 is hungry.Philosopher 0 is hungry.

Philosopher 2 starts eating. 


Philosopher 0 is hungry.
Philosopher 0 starts eating. 
Philosopher 4 is hungry.Philosopher 3 is hungry.
Philosopher 3 starts eating. 
Philosopher 2 is hungry.
Philosopher 2 swaps forks.

Philosopher 1 is hungry.
Philosopher 2 finishes eating and leaves to think.
Philosopher 0 finishes eating and leaves to think.
Philosopher 1 starts eating. 
Philosopher 3 finishes eating and leaves to think.
Philosopher 2 swaps forks.
Philosopher 4 starts eating. 
Philosopher 0 is hungry.Philosopher 1 finishes eating and leaves to think.

Philosopher 2 starts eating. 
Philosopher 3 is hungry.Philosopher 4 finishes eating and leaves to think.

Philosopher 0 starts eating. 
Now we're finishing.
Philosopher 1 is hungry.
Philosopher 4 is hungry.Philosopher 2 finishes eating and leaves to think.

Philosopher 3 starts eating. 
Ph

In [ ]:
import threading
import random
